# Not yet implemented
- Notebook for ideas and functions related to cleaning the quarterly statements data

In [ ]:
balance = pd.read_csv('./data/raw/balance.csv').convert_dtypes()

Remove duplicates on selected columns

In [ ]:
balance = pd.read_csv('./data/raw/balance.csv').convert_dtypes()
dup_features = [
    "cik", 
    "calendarYear", 
    "totalAssets",
    "totalLiabilities",
    "totalEquity",
    ]
balance_clean = balance.drop_duplicates(subset = dup_features, keep='first')
balance_clean = balance_clean.reset_index(drop=True)

Manualy fix some periods

In [ ]:
mask = balance_clean.period == 'Q'
balance_clean.loc[mask, 'period'] = np.nan
balance_clean['period'] = balance_clean.period.str.replace('Q', '')
balance_clean.period = balance_clean.period.astype(pd.Int64Dtype())

Rename columns

In [ ]:
col_rename = {
    'fillingDate':'filingDate',
    'date':'reportDate',
    'period':'periodAPI',
}
balance_clean = balance_clean.rename(columns=col_rename)

join Balance and SEC admission dataframes to get the form columns to the balance sheet

In [ ]:
# reset_indexes to keep track
balance_clean = balance_clean.reset_index(drop=True)
sec = sec.reset_index(drop=True)

1st. join.
- reportDate, symbol, cik

In [ ]:
joincols = ['reportDate', 'cik']
secols = ['form']
# first join
first_join = balance_clean.merge(
    sec.loc[:, joincols + secols], 
    how='left', 
    on = joincols,
    )
mask = first_join.form.isna()
to_join = first_join[mask]
result_1 = first_join[~mask]
result_1.shape, to_join.shape

((43590, 55), (3841, 55))

2nd join.
- filingDate, symbol, cik

In [ ]:
joincols = ['filingDate', 'cik']
secols = ['form']
to_join = to_join.drop(columns=secols)
# first join
second_join = to_join.merge(
    sec.loc[:, joincols + secols], 
    how='left', 
    on = joincols,
    )
mask = second_join.form.isna()
to_join = second_join[mask]
result_2 = second_join[~mask]
result_2.shape, to_join.shape

((333, 55), (3508, 55))

In [ ]:
balance_clean.shape[0] - sec.shape[0]

2205

missing around 2400 vals, OK

In [ ]:
balance_clean = pd.concat([result_1, second_join])
balance_clean.head(2)

,reportDate,symbol,reportedCurrency,cik,filingDate,acceptedDate,calendarYear,periodAPI,cashAndCashEquivalents,shortTermInvestments,...,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,link,finalLink,form
0,2021-09-30,MMM,USD,66740,2021-10-26,2021-10-26 14:53:46,2021,3,4878000000.0,855000000.0,...,48208000000.0,70000000.0,14530000000.0,48278000000.0,0.0,19007000000.0,14129000000.0,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,10-Q
1,2021-06-30,MMM,USD,66740,2021-07-27,2021-07-27 14:55:50,2021,2,4695000000.0,805000000.0,...,48239000000.0,68000000.0,14516000000.0,48307000000.0,0.0,19119000000.0,14424000000.0,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,10-Q


EXPLORE: missing values
- Most of them are because the sec submissions API is missing older filings.
    - Checking if there is an issue with my gathering function

DONE
- There are no issues with the function, the problem is some companies changed their symbols and cik in the sec, such as google before 2015 was named GOOGLE .inc and its cik was 1288776

Since we cannot trust the API symbol,year,period nomenclature so we use the K from from the SEC as guidance and from there we expand it in all directions

In [ ]:
# set the column to infer as nan
try:
    balance_clean.insert(0, 'infered_sec', np.nan)
except:
    pass
# sort
balance_clean = balance_clean.sort_values(['symbol','reportDate'])
# do 10 times
for i in range(10):
    # init balance_clean on 4 if the form is a 10K statement
    cond = balance_clean.form.isin(['10-K', '10-K405', '10-KT'])
    balance_clean['infered_sec'] = balance_clean['infered_sec'].mask(cond= cond, other=4)
    # expand values vertically and horizontally
    for n in range(1,5):
        # only overwrite if its a nan
        nullcond = balance_clean.infered_sec.isna()
        # move the 4 within group and make mask
        cond1 = balance_clean.groupby('symbol').infered_sec.shift(n).eq(4)
        # set to value n if shifted mask & nan
        balance_clean['infered_sec'].mask(cond=(cond1 & nullcond), other=n, inplace = True)
        # shift same for the other direction
        cond2 = balance_clean.groupby('symbol').infered_sec.shift(-n).eq(4)
        if n != 4:
            balance_clean['infered_sec'].mask(cond=(nullcond & cond2), other=(4-n), inplace = True)
        else:
            balance_clean['infered_sec'].mask(cond=(nullcond & cond2), other=4, inplace = True)

In [ ]:
balance_clean.infered_sec.isna().sum()

927

Estimar los periodos a partir de la media de periodo, quitar los simbolos en los que ha habido un T y comprobar si hay algun form K con periodo no 4

In [ ]:
dates_processing(periods,'reportDate', 'filingDate')
# get the dictionary containing the circular mean by period and symbol
dict_means_date = grouped_circmean(periods, 'reportDate_yday', low=0, high=365, levels = ['symbol', 'infered_sec'])
infered_period_date = periods.apply(lambda x: infer_period(x['symbol'], x['reportDate_yday'], dict_means_date), axis=1)
dict_means_fillingDate = grouped_circmean(periods, 'filingDate_yday', low=0, high=365, levels = ['symbol', 'infered_sec'])
infered_period_filling_date = periods.apply(lambda x: infer_period(x['symbol'], x['filingDate_yday'], dict_means_fillingDate), axis=1)

In [ ]:
periods['infered_reportDate'] = infered_period_date
periods['infered_filingDate'] = infered_period_filling_date

In [ ]:
cond1 = periods.infered_sec != periods.infered_reportDate
cond2 = periods.infered_sec != periods.infered_filingDate
supercond = cond1 & cond2
subset = periods[supercond]

In [ ]:
subset.symbol.unique()

array(['AAPL', 'ADM', 'AEP', 'APD', 'ATVI', 'BKR', 'BLL', 'BXP', 'CB',
       'CSCO', 'DAL', 'DG', 'GD', 'GPN', 'HCA', 'IT', 'JCI', 'KIM', 'KR',
       'LEN', 'LHX', 'MOS', 'MTB', 'ODFL', 'OKE', 'ROK', 'ROP', 'SEDG',
       'STZ', 'SWK', 'SWKS', 'TTWO', 'UAL', 'VFC', 'WAB'], dtype=object)

If a symbol had a 10-KT form or 10-QT form, it means it had altered its fiscal year, and the period inference method is not significant

In [ ]:
# get all the observations whose form is either 10-KT or 10-QT
mask_form = periods.form.isin(['10-KT', '10-QT'])
# get their symbols
symbols = periods[mask_form].symbol
# remove them from the subset
subset = subset[~subset.symbol.isin(symbols)]

For the remaining subset keep the estimated values

In [ ]:
periods.loc[subset.index,'infered_sec'] = periods.loc[subset.index,'infered_reportDate']

Now that the sec submissions df is mostly clean we join it with the balance sheet. Because all data sources are dirty, we're left joining the balance sheet and consecuently update the resulting DataFrame to minimize the amount of observations considered null because of dirty features.

In [ ]:
balance_clean['reportDate'] = pd.to_datetime(balance_clean['reportDate'])
balance_clean['filingDate'] = pd.to_datetime(balance_clean['filingDate'])

In [ ]:
balance_clean = dates_processing(balance_clean, 'reportDate', 'filingDate')

In [ ]:
periods.columns

Index(['filingDate', 'reportDate', 'symbol', 'cik', 'form', 'infered_sec',
       'reportDate_int', 'reportDate_month', 'reportDate_year',
       'reportDate_yday', 'filingDate_int', 'filingDate_month',
       'filingDate_year', 'filingDate_yday', 'infered_reportDate',
       'infered_filingDate'],
      dtype='object')

In [ ]:
balance_clean.columns

Index(['reportDate', 'symbol', 'reportedCurrency', 'cik', 'filingDate',
       'acceptedDate', 'calendarYear', 'period', 'cashAndCashEquivalents',
       'shortTermInvestments', 'cashAndShortTermInvestments', 'netReceivables',
       'inventory', 'otherCurrentAssets', 'totalCurrentAssets',
       'propertyPlantEquipmentNet', 'goodwill', 'intangibleAssets',
       'goodwillAndIntangibleAssets', 'longTermInvestments', 'taxAssets',
       'otherNonCurrentAssets', 'totalNonCurrentAssets', 'otherAssets',
       'totalAssets', 'accountPayables', 'shortTermDebt', 'taxPayables',
       'deferredRevenue', 'otherCurrentLiabilities', 'totalCurrentLiabilities',
       'longTermDebt', 'deferredRevenueNonCurrent',
       'deferredTaxLiabilitiesNonCurrent', 'otherNonCurrentLiabilities',
       'totalNonCurrentLiabilities', 'otherLiabilities',
       'capitalLeaseObligations', 'totalLiabilities', 'preferredStock',
       'commonStock', 'retainedEarnings',
       'accumulatedOtherComprehensiveIncomeLos

In [ ]:
balance_clean.shape

(47637, 62)

In [ ]:
(joined_data.infered_sec != joined_data.period).sum()

4927

In [ ]:
joined_data['infered_sec'] = np.nan
joined_data = joined_data.sort_values(['symbol','reportDate'])
# init joined_data on 4 if the form is a 10K statement
cond = joined_data.form.isin(['10-K', '10-K405', '10-KT'])
joined_data['infered_sec'] = joined_data['infered_sec'].mask(cond= cond, other=4)
# depth of 2
for i in range(2):
    # expand values vertically and horizontally
    for n in range(1,5):
        # only overwrite if its a nan
        nullcond = joined_data.infered_sec.isna()
        # move the 4 within group and make mask
        cond1 = joined_data.groupby('symbol').infered_sec.shift(n).eq(4)
        # set to value n if shifted mask & nan
        joined_data['infered_sec'].mask(cond=(cond1 & nullcond), other=n, inplace = True)
        # shift same for the other direction
        cond2 = joined_data.groupby('symbol').infered_sec.shift(-n).eq(4)
        if n != 4:
            joined_data['infered_sec'].mask(cond=(nullcond & cond2), other=(4-n), inplace = True)
        else:
            joined_data['infered_sec'].mask(cond=(nullcond & cond2), other=4, inplace = True)

,reportDate,symbol,reportedCurrency,cik,filingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,reportDate_int,reportDate_month,reportDate_year,reportDate_yday,filingDate_int,filingDate_month,filingDate_year,filingDate_yday,form,infered_sec
33919,2021-09-30,OGN,USD,1821825,2021-11-12,2021-11-12 16:52:11,2021,3,1008000000.0,0.0,...,20210930,9,2021,273,20211112,11,2021,316,10-Q,NaN
33920,2021-06-30,OGN,USD,1821825,2021-08-16,2021-08-16 08:20:13,2021,2,730000000.0,0.0,...,20210630,6,2021,181,20210816,8,2021,228,10-Q,NaN
33921,2021-03-31,OGN,USD,1821825,2021-06-21,2021-06-21 08:16:50,2021,1,141000000.0,0.0,...,20210331,3,2021,90,20210621,6,2021,172,10-Q,NaN
20,1992-12-31,ABMD,USD,815094,1992-12-31,1992-12-31,1993,3,3500000.0,0.0,...,19921231,12,1992,365,19921231,12,1992,365,NaN,NaN
21,1992-09-30,ABMD,USD,815094,1992-09-30,1992-09-30,1993,4,3200000.0,0.0,...,19920930,9,1992,274,19920930,9,1992,274,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4032,2003-09-30,WRK,USD,1732845,2003-09-30,2003-09-30,2003,4,14173000.0,0.0,...,20030930,9,2003,273,20030930,9,2003,273,NaN,NaN
4033,2003-06-30,WRK,USD,1732845,2003-06-30,2003-06-30,2003,3,8205000.0,0.0,...,20030630,6,2003,181,20030630,6,2003,181,NaN,NaN
4034,2003-03-31,WRK,USD,1732845,2003-03-31,2003-03-31,2003,2,5422000.0,0.0,...,20030331,3,2003,90,20030331,3,2003,90,NaN,NaN
4035,2002-12-31,WRK,USD,1732845,2002-12-31,2002-12-31,2003,1,6803000.0,0.0,...,20021231,12,2002,365,20021231,12,2002,365,NaN,NaN


In [ ]:
fred_daily = df.reset_index()
fred_daily.columns = real_names 

In [ ]:
# FRED SERIES
##########################################
# Daily indicators
series_names = ['DGS3MO', 
                'DGS1', 
                'DGS2', 
                'DGS3', 
                'DGS5', 
                'DGS10', 
                'DGS20', 
                'DGS30', 
                'DCOILWTICO', 
                'DCOILBRENTEU', 
                'VIXCLS',
                'GOLDAMGBD228NLBM']
real_names = ['Date',
              'T3m', 
              'T1y', 
              'T2y', 
              'T3y', 
              'T5y', 
              'T10y', 
              'T20y', 
              'T30y',
              'WTI', 
              'BRENT', 
              'VIX', 
              'GOLD']

start = datetime.datetime(1980, 1, 1)
end = datetime.date.today()

for i, name in enumerate(series_names):
    if i == 0:
        df = web.DataReader(name, data_source = 'fred', start=start, end = end)
    else:
        series = web.DataReader(name, data_source = 'fred', start=start, end = end)
        df = pd.concat([df, series], axis = 1)

fred_daily = df.reset_index()
fred_daily.columns = real_names 

# Exploration

In [ ]:
cflow = pd.read_csv('../data/raw/cflow_y.csv', index_col = 0)
income = pd.read_csv('../data/raw/income_y.csv', index_col = 0)
balance = pd.read_csv('../data/raw/balance_y.csv', index_col = 0)

## 10K Statements

special interest in period, symbol and calendarYear

In [ ]:
mycols = [
    'period', 
    'symbol', 
    'calendarYear'
    ]
balance[mycols].isna().sum()

period          1
symbol          0
calendarYear    0
dtype: int64

In [ ]:
balance[mycols].nunique()

period            5
symbol          505
calendarYear     38
dtype: int64

duplicate observations??

In [ ]:
# features to find duplicates
dup_features = [
    "symbol", 
    "calendarYear", 
    "totalAssets",
    "totalLiabilities", 
    "totalEquity",
    ]
mask_cols = dup_features + ["date", "fillingDate", "period"]
df_duplicates = check_duplicates(balance, dup_features).loc[:,mask_cols]


In [ ]:
df_duplicates

,symbol,calendarYear,totalAssets,totalLiabilities,totalEquity,date,fillingDate,period
745,AMD,2013,3797000000.0,3382000000.0,415000000.0,2013-03-31,2013-05-06,Q1
746,AMD,2013,3797000000.0,3382000000.0,415000000.0,2013-03-30,2013-05-06,Q1
808,AMD,1997,3426000000.0,1382000000.0,2044000000.0,1997-09-28,1997-11-12,Q3
810,AMD,1997,3426000000.0,1382000000.0,2044000000.0,1997-03-30,1997-05-14,Q1
1011,AES,1996,3622000000.0,2688000000.0,934000000.0,1996-12-31,1996-12-31,Q4
1014,AES,1996,3622000000.0,2688000000.0,934000000.0,1996-03-31,1996-03-31,Q4
2247,MO,2021,48776000000.0,45781000000.0,2995000000.0,2021-03-31,2021-04-29,Q1
2248,MO,2021,48776000000.0,45781000000.0,2995000000.0,2021-03-01,2021-04-29,Q2
2581,AAL,2021,68437000000.0,75874000000.0,-7437000000.0,2021-09-30,2021-10-21,Q3
2582,AAL,2021,68437000000.0,75874000000.0,-7437000000.0,2021-09-03,2021-10-21,Q4


In [ ]:
mask = (balance.period.isna()) | (balance.period == 'Q')
balance[mask]

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,link,finalLink
4794,2010-02-04,ADM,USD,7084,2010-02-08,2010-02-05 19:10:55,2010,Q,1317000000.0,317000000.0,...,14744000000.0,23878000000.0,5987000000.0,20731000000.0,29865000000.0,3661000000.0,732000000.0,-585000000.0,https://www.sec.gov/Archives/edgar/data/7084/0...,https://www.sec.gov/Archives/edgar/data/7084/0...
9479,2001-09-30,CBRE,USD,1138118,2001-11-14,2001-11-14 00:00:00,2001,<NA>,24211000.0,0.0,...,247463000.0,1219638000.0,0.0,247463000.0,1219638000.0,0.0,639664000.0,615453000.0,https://www.sec.gov/Archives/edgar/data/113811...,https://www.sec.gov/Archives/edgar/data/113811...


In [ ]:
print(balance.loc[mask, mask_cols].to_markdown())

|      | symbol   |   calendarYear |   totalAssets |   totalLiabilities |   totalEquity | date       | fillingDate   | period   |
|-----:|:---------|---------------:|--------------:|-------------------:|--------------:|:-----------|:--------------|:---------|
| 4794 | ADM      |           2010 |   2.9865e+10  |        9.134e+09   |   2.0731e+10  | 2010-02-04 | 2010-02-08    | Q        |
| 9479 | CBRE     |           2001 |   1.21964e+09 |        9.72175e+08 |   2.47463e+08 | 2001-09-30 | 2001-11-14    | <NA>     |


Are there any observations where, for a given symbol and year, does not have 4 periods?? First and last year excluded

In [ ]:
# get min and max from each group and filter them out
min_year = balance.groupby('symbol')['calendarYear'].transform('min')
max_year = balance.groupby('symbol')['calendarYear'].transform('max')
filter = (balance.calendarYear == min_year) | (balance.calendarYear == max_year)
# get nunique periods for each calendar year excluding min and max years
aux = balance[~filter].groupby(['symbol', 'calendarYear']).agg({'period': ['nunique']}).reset_index()
aux.columns = ('symbol', 'calendarYear', 'period_nunique')
# show in balance the values which dont have 4 unique periods per year
df_incomplete = aux[aux.period_nunique!=4]
keys = tuple(zip(df_incomplete.symbol, df_incomplete.calendarYear))
# creating the mask
mask = False
for symbol, year in keys:
    mask = mask | ((balance.symbol==symbol) & (balance.calendarYear==year))
# dataframe with all the observations
df_incomplete = balance.loc[mask, :].sort_values(by=['symbol', 'calendarYear', 'period'])


In [ ]:
balance.loc[[902,903,904], mask_cols]

,symbol,calendarYear,totalAssets,totalLiabilities,totalEquity,date,fillingDate,period
902,AAP,2004,2063000000.0,1374000000.0,689000000.0,2004-04-24,2004-05-21,Q1
903,AAP,2004,1983000000.0,1352000000.0,631000000.0,2004-01-03,2004-03-12,Q4
904,AAP,2003,2011000000.0,1414000000.0,597000000.0,2003-10-04,2003-11-06,Q3


In [ ]:
numerical = balance.select_dtypes(include=['int', 'float']).columns
explore_numerical(balance, numerical)

,nulls,nulls_pct,min,max,mean,median,std,var
cik,0,0.0,1.800000e+03,1.841666e+06,6.807326e+05,8.150940e+05,5.171915e+05,2.674871e+11
calendarYear,0,0.0,1.985000e+03,2.022000e+03,2.008682e+03,2.009000e+03,7.862905e+00,6.182528e+01
cashAndCashEquivalents,0,0.0,-3.130000e+07,7.598690e+11,3.144459e+09,4.350220e+08,1.764755e+10,3.114361e+20
shortTermInvestments,0,0.0,-4.284000e+09,4.830690e+11,3.485959e+09,0.000000e+00,2.368985e+10,5.612088e+20
cashAndShortTermInvestments,0,0.0,-2.874000e+09,1.026474e+12,6.630419e+09,5.738955e+08,3.593816e+10,1.291552e+21
netReceivables,0,0.0,-1.418230e+08,4.538370e+11,3.238153e+09,6.843595e+08,1.393233e+10,1.941097e+20
inventory,0,0.0,-1.783500e+07,8.696100e+10,1.143456e+09,2.200000e+08,3.181732e+09,1.012342e+19
otherCurrentAssets,0,0.0,-2.264398e+17,1.499626e+13,-7.962790e+12,2.000500e+08,1.252817e+15,1.569551e+30
totalCurrentAssets,0,0.0,-2.264398e+17,1.499650e+13,-7.952944e+12,2.375700e+09,1.252817e+15,1.569551e+30
propertyPlantEquipmentNet,0,0.0,-1.457000e+07,3.449770e+11,6.631450e+09,1.244388e+09,1.816347e+10,3.299115e+20


### Period Inconsistencies
1. Misnomer periods
<br>

|      | symbol   |   calendarYear |   totalAssets |   totalLiabilities |   totalEquity | date       | fillingDate   | period   |
|-----:|:---------|---------------:|--------------:|-------------------:|--------------:|:-----------|:--------------|:---------|
| 4794 | ADM      |           2010 |   2.9865e+10  |        9.134e+09   |   2.0731e+10  | 2010-02-04 | 2010-02-08    | Q        |
| 9479 | CBRE     |           2001 |   1.21964e+09 |        9.72175e+08 |   2.47463e+08 | 2001-09-30 | 2001-11-14    | <NA>     |

2. Periods carry to the next year as in index=903
<br>

|     | symbol   |   calendarYear |   totalAssets |   totalLiabilities |   totalEquity | date                | fillingDate         | period   |
|----:|:---------|---------------:|--------------:|-------------------:|--------------:|:--------------------|:--------------------|:---------|
| 902 | AAP      |           2004 |     2.063e+09 |          1.374e+09 |      6.89e+08 | 2004-04-24 00:00:00 | 2004-05-21 00:00:00 | Q1       |
| 903 | AAP      |           2004 |     1.983e+09 |          1.352e+09 |      6.31e+08 | 2004-01-03 00:00:00 | 2004-03-12 00:00:00 | Q4       |
| 904 | AAP      |           2003 |     2.011e+09 |          1.414e+09 |      5.97e+08 | 2003-10-04 00:00:00 | 2003-11-06 00:00:00 | Q3       |

2. Duplicates rows with different periods
<br>

|      | symbol   |   calendarYear |   totalAssets |   totalLiabilities |   totalEquity | date                | fillingDate         | period   |
|-----:|:---------|---------------:|--------------:|-------------------:|--------------:|:--------------------|:--------------------|:---------|
| 2581 | AAL      |           2021 |    6.8437e+10 |         7.5874e+10 |    -7.437e+09 | 2021-09-30 00:00:00 | 2021-10-21 00:00:00 | Q3       |
| 2582 | AAL      |           2021 |    6.8437e+10 |         7.5874e+10 |    -7.437e+09 | 2021-09-03 00:00:00 | 2021-10-21 00:00:00 | Q4       |



